In [1]:
using LinearAlgebra
using Distributions
using BenchmarkTools

function randmatrix(n::Int64, m::Int64)
    p = Float64(m)/Float64(n)^2 #Calculates success rate
    M = rand(Bernoulli(p), n, n) #Creates n×n bernoulli matrix (0 and 1s)
    N = rand(Uniform(-1,1), n, n) #Creates n×n matrix with values random between -1 and 1
    for i = 1:n
        for j = 1:n
            if M[i,j] == 0
                N[i,j] = 0.0 #If value in M is 0, corresponding value in N is changed to 0.0, otherwise, do nothing
            end
        end
    end
    return N
end

function gramschmidt(A::Matrix{Float64})
    B = copy(A) #Create output matrix
    m = size(A,1) #Extract Dimensions of matrix
    n = size(A,2)
    for i = 1:m #We will run this for every column vector
        veci = B[:,i] #Extact ith vector
        uveci = veci ./ norm(veci) #Normalise ith vector
        B[:,i] = uveci #Update B to contain normalised vector
        for j = i+1:m #We will update all vectors after the ith
            vecj = B[:,j] #Extract jth vector
            B[:,j] = vecj - (vecj⋅veci).*uveci./norm(veci) #We substract the vector projection of vecj on veci from vecj
        end
    end
    return B #Return matrix B
end

function qrdecomp(A::Matrix{Float64})
    Q = gramschmidt(A) #Calculates Q using gram-schmidt algorithm
    R = transpose(Q)*A #Calculates R
    return Q, R #Returns the two matrices
end

function qralgorithm(A::Matrix{Float64})
    n = size(A,1) #Gets dimension of matrix
    ϵ = 1e-16 #Error term (used to check when algorithm should terminate)
    Q, R = qrdecomp(A) #Calculates QR form of matrix A 
    newA = R*Q #Calculates next A in sequence by reversing order of QR form
    newQ, newR = qrdecomp(newA) #Calculates QR form of this new matrix
    while all(diag(abs.(newA - A) .< ϵ)) == false #checks if the diagonal (eigenvalues) of the newA and A are all
                                                  #within ϵ of each other and continues iterating if they are not
        A, Q, R = newA, newQ, newR #Updates A, Q, and R to be new values
        newA = R*Q #Calculates next A in sequence by reversing order of QR form
        newQ, newR = qrdecomp(newA) #Calculates QR form of this this new matrix     
    end
    Eigen = diag(newA) #Generates list of eigenvectors of orginal matrix (they are the diagonals of newA)
    return Eigen #Returns eigenvalues
end

qralgorithm (generic function with 1 method)

In [ ]:
o = []
time = []
for i in 2:100                                            #Set the n-size of matrix
    print("Processing ",i,"x",i)                          #Shows progress of fun
    a = randmatrix(i,i)                                   #Generate random matrix
    t = @benchmark qralgorithm($a) samples=10 evals=1
    push!(o,i)                                            #Fill the list with n-values for x-axis plotting
    push!(time, mean(t).time)                             #Fill the list with mean runtime of function for given n-size matrix
end

In [ ]:
using Plots
plot(o,time)